# Приложение: проверка статистических гипотез

* *[Введение](#Введение)*  
* *[Проверка статистической гипотезы](#Проверка-статистической-гипотезы)*  
* *[Сравнение средних в группах](#Сравнение-средних-в-группах)*
* *[Особенности интерпретации](#Особенности-интерпретации)*
* *[Заключение](#Заключение)*
* *[Ссылки](#Ссылки)*

# Введение

Для анализа А/Б-тестов используют метод проверки статистических гипотез [[StTest](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing)]. Чаще всего в таком подходе предполагают, что между вариантами нет разницы, после чего смотрят, насколько такое предположение объясняет экспериментальные данные. Если вероятность получить данные мала, то считается, что предположение можно отвергнуть и между группами есть значимая разница.  

Вначале описывается терминология и приводится пример проверки статистической гипотезы. Далее рассматривает сравнение средних двух распределений. Затем обсуждаются особенности интерпретации этого метода.

# Проверка статистической гипотезы

Есть экспериментальные данные $data$. Есть гипотеза $H$ об этих данных. Есть случайная величина $T$, распределение которой $P_{T}(x | H)$ в предположении $H$ известно - такие величины служат основой одноименных "статистических тестов". По фактическим данным считается "тестовая статистика"  $x_{data}$ [[TestStat](https://en.wikipedia.org/wiki/Test_statistic)]. Считается вероятность получить "фактическое или более экстремальное" значение "тестовой статистики". Эту вероятность называют "$p$-значением" [[PVal](https://en.wikipedia.org/wiki/P-value)]. В зависимости от контекста $p = P_{T}(x \ge x_{data} | H)$, $p = P_{T}(x \le x_{data} | H)$ или $p = P_{T}(|x - x_{data}| \ge 0 | H)$ [[TailedTests](https://en.wikipedia.org/wiki/One-_and_two-tailed_tests)]. Если вероятность "достаточно мала", гипотезу $H$ "отвергают", если "не достаточно мала" - "принимают".  

Обсуждение такого подхода приводится в разделе ["Особенности интерпретации"](#Особенности-интерпретации).

В качестве примера можно рассмотреть проверку равенства вероятностей выпадения орла и решки по нескольким броскам монеты [[FairCoin](https://en.wikipedia.org/wiki/Checking_whether_a_coin_is_fair)]. Ситуация следующая:
- данные: в 100 бросках монетки в 60 случаях выпал орел.  
- гипотеза: вероятность выпадения орла 0.5   
- случайная величина с известным распределением: вероятность k успехов в N попытках при вероятности $p$ успеха в одной попытке можно моделировать биномиальным распределением $Binom(p; k, N)$  
- тестовая статистика: число успехов k=60 в общих попытках N=100
- $p$-значение: вероятность получить 60 или больше успехов в 100 попытках $p = \sum_{i \ge 60}Binom(0.5; i, 100) = 1 - CDF_{Binom}(0.5; 60, 100)$
- "достаточно малую" вероятность обычно выбирают на уровне 5%, т.е. если $p < 0.05$, то гипотезу $H$ считают неверной.  

Ниже приведен график распределения тестовой статистики $Binom(0.5; k, 100)$, выделены "фактические или более экстремальные" значения $k$, проведены расчеты $p$-значения. $P$-значение оказывается равным 0.018. Это меньше 0.05, что дает основания считать гипотезу о вероятности выпадения орла 50\% неверной.  

In [ ]:
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go

np.random.seed(7)

In [ ]:
N = 100
s = 60
p = 0.5
alpha = 0.05

x = np.arange(0, N+1)
y = stats.binom.pmf(p=p, k=x, n=N)
col = ['blue' if x < s else 'red' for x in x]

fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=y, marker_color=col))
fig.update_layout(
    title=f"Probability of X heads in N={N} coin flips for a fair coin",
    height=450, width=800
)
fig.show()

pval = 1 - stats.binom.cdf(p=p, k=s, n=N)
print(f"p value = P(s >= {s} | H) = {pval:.3f}")
print(f"H {'can' if pval < alpha else 'can not'} be rejected")

Даже если гипотеза верна, в эксперименте можно получить такие данные, что нужно принять решение об отклонении гипотезы. Вероятность получить данные по которым нужно принять решение "отвергнуть гипотезу" при условии, что гипотеза верна, называют вероятностью ошибки первого рода и обозначают $\alpha$. Эту величину также называют "статистической значимостью". Величина $1-\alpha$ характеризует вероятность принять корректное решение оставить гипотезу, при условии, что она верна. Вероятность получить данные, по которым нужно "принять гипотезу" при условии, что она не верна, называют вероятностью ошибки второго рода и обозначают $\beta$. Величину $1-\beta$ называют статистической мощностью [[StErrors](https://en.wikipedia.org/wiki/Type_I_and_type_II_errors), [StSign](https://en.wikipedia.org/wiki/Statistical_significance), [StPower](https://en.wikipedia.org/wiki/Power_of_a_test)]. 

\begin{align}
\mbox{Статистическая значимость и ошибка первого рода: }& P(\mbox{Решение отклонить }H | H) = \alpha,
\\
\mbox{Корректное решение оставить H: }& P(\mbox{Решение принять }H | H) = 1 - \alpha .
\\
\\
\mbox{Ошибка второго рода: }& P(\mbox{Решение принять }H | \neg H) = \beta, 
\\
\mbox{Статистическая мощность: }& P(\mbox{Решение отклонить }H | \neg H) = 1 - \beta .
\end{align}

Статистическая значимость и мощность не полностью определяют вероятность выполнения гипотезы $H$. Остается неизвестным, верна в реальности гипотеза $H$ или нет. Подробнее см. раздел ["Особенности интерпретации"](#Особенности-интерпретации).

# Сравнение средних в группах

Метод проверки статистических гипотез применяется в А/Б-тестах для сравнения групп.  

Группы можно сравнивать по разным метриками. Ниже рассматривается сравнение средних.
Т.е. по сэмлам из двух групп нужно оценить средние в распределениях и выбрать вариант с "лучшим" средним.

Для сравнения групп смотрят на выборочные средние $\mu_{A_n}$ и $\mu_{B_n}$. Проверяют гипотезу, что группы не отличаются - нулевую гипотезу $H_0$ [[HNull](https://en.wikipedia.org/wiki/Null_hypothesis)]. У выборочных средних среднее совпадает со средним исходного распределения, стандартное отклонение определяется стандартной ошибкой среднего [[StdErr](https://en.wikipedia.org/wiki/Standard_error)]. Если выборка "достаточно большая", то в силу центральной предельной теоремы [[CLT](https://en.wikipedia.org/wiki/Central_limit_theorem)] их распределение приближенно можно считать нормальным. Ниже приведены соотношения для $\mu_{A_n}$, для $\mu_{B_n}$ соотношения аналогичные.

$$
\mu_{A_n} = \frac{1}{N_A} \sum_i A_i,
\\
E[\mu_{A_n}] = E[A],
\quad
\sigma_{\mu_{A_n}} = \frac{\sigma_A}{\sqrt{N_A}} \approx \frac{s_A}{\sqrt{N_A}},
\quad
s^2_{A} = \frac{1}{N_A - 1} \sum_i (A_i - E[A])^2,  
\\
\mu_{A_n} \sim Norm(E[A], \sigma^2_{\mu_{A_n}}) .
$$

Разность нормальных распределений также нормальное распределение. Поэтому разность выборочных средних также можно приближенно считать распределенной нормально 

$$
\mu_{\Delta} = \mu_{A_n} - \mu_{B_n},
\\
E[\mu_{\Delta}] = E[\mu_{A_n}] - E[\mu_{B_n}] = E[A] - E[B],
\quad
\sigma_{\Delta} \approx s_{\Delta} = \sqrt{s_{\mu_{A_n}}^2 + s_{\mu_{B_n}}^2},
\\
\mu_{\Delta} \sim Norm(E[\mu_{\Delta}], \sigma^2_{\Delta}).
\\
$$


В качестве "случайной величины, распределение которой в предположении нулевой гипотезы известно", выбирают 

$$
t = \frac{\mu_{\Delta} - E[\mu_{\Delta}]}{s_{\Delta}}. 
$$

Если верна нулевая гипотеза, т.е. между группами нет разницы, то $E[\mu_{\Delta}] = 0$, а $\mu_{\Delta}$ и $s_{\Delta}$ можно вычислить из сэмплов. При "достаточно большом" размере выборки распределение величины $t$ можно приближенно считать нормальным 

$$
t \sim Norm(0, 1).
$$

Использование нормального распределения $Norm(0, 1)$ для $\mu_{\Delta}/s_{\Delta}$ оправдано при "достаточно большом" размере выборки. Чаще используют $t$-распределение $\mu_{\Delta}/s_{\Delta} \sim t_{n-1}$ [[TDist](https://en.wikipedia.org/wiki/Student%27s_t-distribution)] или приближение Уэлча [[WelchTTest](https://en.wikipedia.org/wiki/Welch%27s_t-test)]. В таком случае подход называют $t$-тестом [[TTest](https://en.wikipedia.org/wiki/Student's_t-test)].

*Использование нормального распределения $Norm(0, 1)$ для $\mu_{\Delta}/s_{\Delta}$ оправдано при "достаточно большом" размере выборки. Более точным может быть приближение на основе $t$-распределения [[TDist](https://en.wikipedia.org/wiki/Student%27s_t-distribution)]. Для сэмпла случайной величины $X$ рассматривают $\sqrt{n} (\mu_{X_n} - E[X])/s_n$, где $\mu_{X_n}$ - выборочное среднее, $E[X]$ - неизвестное точное среднее, $s_n^2$ - несмещенная оценка дисперсии. Можно записать в виде $t = N / \sqrt{\chi^2_{n-1}}$, $N = (\mu_{X_n} - E[X])/\sigma$, $\chi^2_{n-1}=s^2_n/\sigma^2$, где $\sigma^2$-неизвестное точное значение дисперсии. По мере роста выборки $N$ и $\chi^2$ будут стремиться к нормальному и хи-квадрат распределениям. Отношение нормального распределения к корню из хи-квадрат распределения $N/\sqrt{\chi^2}$ задается $t$-распределением [[TDist](https://en.wikipedia.org/wiki/Student%27s_t-distribution)]. Поэтому отношение $(\mu_x - E[x])/s$ приближенно описывается $t$-распределением, а подход называют $t$-тестом [[TTest](https://en.wikipedia.org/wiki/Student's_t-test)]. Особенность $t$-статистики в том, что она не зависит от неизвестных точных среднего и дисперсии - только от их оценок из сэмплов. Нормальное распределение является предельным случаем $t$-распределения, поэтому работает приближенный метод выше. Описание выше полагается на центральну предельную теорему для нормальности. Т.к. интересует оценка $(\mu_{\Delta} - E[x])/s_{\Delta}$ для разности двух случайных величин, то возникает проблема определения $\mu_{\Delta}$ и $s_{\Delta}$ из сэмплов. Используют приближение Уэлча [[WelchTTest](https://en.wikipedia.org/wiki/Welch%27s_t-test)].* 

На конкретных данных пример выглядит следующим образом. Допустим, есть две монетки с разными вероятностями выпадения орла (две серии Бернулли). Нужно выбрать монетку с большей вероятностью. Пусть $N_A, N_B, s_A, s_B$ - количество бросков и орлов в группах $A$ и $B$.

При проведении тестов для контроля за уровнями статистической значимости и мощности размер выборки расчитывают заранее. Выбирают статистическую значимость $\alpha$, мощность $\beta$, базовую конверсию, минимальную величину эффекта, по этим значениям рассчитывают размер выборки $N$ в каждой группе. Для сравнения конверсий с помощью $t$-теста при $\alpha = 0.05$, $\beta = 0.2$, базовой конверсии $p=0.3$, минимальном размере эффекта $\Delta = 0.05$ размер каждой группы должен быть $\sim 15000$ [[MillerABSize](https://www.evanmiller.org/ab-testing/sample-size.html)]. Возможный способ расчета и обсуждение см. в разделе ["Особенности интерпретации"](#Особенности-интерпретации).

На первом графике ниже приведены средние групп и построены нормальные распределения выборочных средних в предположении центральной предельной теоремы. На втором графике приведено нормальное распределение разности выборочных средних в предположении одинаковых групп, обозначено полученное в эксперименте значение разности и выделены области "достаточно малой" вероятности для отвержения предположения о равенстве групп.

In [ ]:
alpha = 0.05

p_a_exact = 0.3
p_b_exact = 0.32

n = 15000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

p_a = sa/n
a_stderr = np.sqrt(p_a * (1 - p_a) / n)

p_b = sb/n
b_stderr = np.sqrt(p_b * (1 - p_b) / n)

x=np.linspace(0, 1, 1000)

col_a = 'red'
col_b = 'blue'

fig = go.Figure()
fig.add_trace(go.Scatter(x=[p_a], y=[stats.norm.pdf(p_a, p_a, a_stderr)],
                         name='A', marker_color=col_a, mode='markers'))
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, p_a, a_stderr),
                         name='CLT A', line_color=col_a, line_dash='dash'))
fig.add_trace(go.Scatter(x=[p_b], y=[stats.norm.pdf(p_b, p_b, b_stderr)],
                         name='B', line_color=col_b, mode='markers'))
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, p_b, b_stderr),
                         name='CLT B', marker_color=col_b, line_dash='dash'))
fig.update_layout(
    title='Assumed CLT-like Means Distributions',
    xaxis_range=[0.2, 0.5],
    height=450, width=800
)
fig.show()


#todo: use one-sided test?
diff = p_b - p_a
diff_stderr = np.sqrt(a_stderr**2 + b_stderr**2)
alpha = 0.05
x_a_l = stats.norm.ppf(alpha/2, loc=0, scale=diff_stderr)
x_a_u = stats.norm.ppf(1-alpha/2, loc=0, scale=diff_stderr)
diff_pval = stats.norm.cdf(0, diff, diff_stderr)

x=np.linspace(-5*diff_stderr, 5*diff_stderr, 1000)

col = 'rgba(0, 0, 250, 0.7)'
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, 0, diff_stderr),
                         line_color=col, name='Diff'))
fig.add_trace(go.Scatter(x=x[x <= x_a_l], y=stats.norm.pdf(x, 0, diff_stderr)[x <= x_a_l], 
                         fill='tozeroy',
                         name='α/2',
                         line_color=col, fillcolor=col))
fig.add_trace(go.Scatter(x=x[x >= x_a_u], y=stats.norm.pdf(x, 0, diff_stderr)[x >= x_a_u], 
                         fill='tozeroy',
                         name='1 - α/2',
                         line_color=col, fillcolor=col))
#fig.add_vline(p_b - p_a)
fig.add_trace(go.Scatter(x=[p_b - p_a, p_b - p_a], y=[0, stats.norm.pdf(0, 0, diff_stderr)],
                        line_dash='dash', mode='lines', name='(pb-pa)'))
fig.update_layout(
    title='Approx Means Difference Given H0 is True',
    height=450, width=800
)
fig.show()

#todo: use alpha/2 instead of alpha?
print(f"p value = {diff_pval:.3f}")
print(f"H {'can' if diff_pval < alpha else 'can not'} be rejected")

$P$-значение "достаточно мало" и согласно методологии проверок гипотез предположение о равенстве групп можно отвергнуть.

Сравнение описанного выше приближенного варианта с $t$-тестом из SciPy [[SciPyT](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)]

In [ ]:
def approx_ttest_conv_pval(sa, na, sb, nb):
    pa = sa / na
    pb = sb / nb
    stderr_a = np.sqrt(pa * (1 - pa) / na)
    stderr_b = np.sqrt(pb * (1 - pb) / nb)
    diff = pb - pa
    diff_stderr = np.sqrt(stderr_a**2 + stderr_b**2)
    pval = stats.norm.cdf(0, diff, diff_stderr)
    return pval

def scipy_ttest_conv_pval(sa, na, sb, nb, **kwargs):
    a = np.zeros(n)
    a[:sa] = 1
    b = np.zeros(n)
    b[:sb] = 1
    t, p = stats.ttest_ind(a, b, equal_var=False, **kwargs)
    return p

p_a_exact = 0.3
p_b_exact = 0.32
n = 15000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

print('scipy.stats.ttest_indep p-val:', scipy_ttest_conv_pval(sa, n, sb, n, alternative='less'))
print('approx p-val:', approx_ttest_conv_pval(sa, n, sb, n))

$P$-значения совпадают в 3 десятичных знаках. В приближенном варианте `approx_ttest_conv_pval` считается односторонний тест, поэтому в `stats.ttest_ind` нужно указывать соответстующий параметр (`alternative='less'`). При двухстороннем тесте (`alternative='two-sided'`) $p$-значение будет в два раза выше.

Описанный подход применим к случайным величинам с произвольным распределением, для которых выполнены условия центральной предельной теоремы. Для скошенных распределений может потребоваться больше данных для ее применимости. Необходимый размер выборки можно оценить с помощью теоремы Берри-Ессеена [[BerEsTheor](https://en.wikipedia.org/wiki/Berry%E2%80%93Esseen_theorem)]. 

Помимо $t$-теста есть другие тесты для сравнения распределений и их свойств: тест Колмогорова-Смирнова для распределений [[KSTest](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test)], U-тест [[UTest](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test)],
сравнение медиан [[MedianTest](https://en.wikipedia.org/wiki/Median_test)]. Они не требуют предположения о нормальных распределениях сравниваемых величин. При этом у них те же особенности интерпретации, что и у $t$-теста - см. далее.

# Особенности интерпретации

В А/Б-тесте нужно ответить на следующие вопросы:

- Какой вариант лучше и насколько?
- Каковы оценки целевой метрики в каждом варианте?
- Насколько уверены в оценке?
- Сколько должен продолжаться эксперимент?

Метод "проверки статистических гипотез" в описанном выше виде не дает на них прямого ответа.

Для ответа на вопрос "Какой вариант лучше?" и "Насколько уверены в оценке?" подойдет вероятность разности средних больше нуля

$$
P(\mu_A - \mu_B > 0 | data) .
$$ 

В методе проверки гипотез решение принимается на основе $p$-значения. Оно считается как вероятность получить "фактические или более экстремальные" данные при выполнении нулевой гипотезы $H_0$

$$
p = P(x \ge x_{data} | H_0) = P(x \ge x_{data} | \mu_A = \mu_B).
$$

$P$-значение отличается от искомой вероятности $P(\mu_A - \mu_B > 0 |data)$. Оно показывает, насколько вероятно получить данные в рамках выбранной гипотезы. При этом его нельзя интерпретировать как вероятность, что гипотеза $H_0$ верна или не верна $P(x \ge x_{data} | H_0) \ne P(H_0 | data)$ [[StatTestMisinterpret](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4877414/)].

Хотя $P(H_0|data) = P(\mu_A = \mu_B | data)$ также отличается от вероятности $P(\mu_A - \mu_B > 0 |data)$, оценку $P(H_0|data)$ можно рассмотреть подробнее. В общем случае

$$
P(data | H_0) \ne P(H_0 | data).
$$ 

Корректное выражение для оценки вероятности нулевой гипотезы на основе собранных данных можно записать с помощью соотношения Байеса

$$
P(H_0 | data) = \frac{P(data | H_0) P(H_0)}{P(data)}
= \frac{P(data | H_0) P(H_0)}{P(data|H_0) P(H_0) + P(data| \neg H_0) P(\neg H_0) } .
$$

Кроме $P(data|H_0)$ в этом соотношении также учитываются $P(H_0)$, $P(data| \neg H_0)$ и $P(\neg H_0)$. Т.е. для выбора между гипотезами нужно сравнивать вероятности получить данные в рамках разных гипотез и распространенность самих гипотез [[UU](https://onlinelibrary.wiley.com/doi/book/10.1002/9781118650158)]. *Принимать решение об отклонении нулевой гипотезы только по малости $p$-значения в абсолютных величинах непоследовательно с точки зрения теории вероятностей - нужно учитывать малость относительно других гипотез и распространненость других гипотез.*

Принятие решения без учета других гипотез будет вариантом ошибки игнорирования базового процента [[BaseRateFal](https://en.wikipedia.org/wiki/Base_rate_fallacy)].

In [ ]:
w = 10
h = 10
h0_w = 2.5/w
alpha = 8/h
beta=2/h

fig = go.Figure()
fig.add_trace(go.Scatter(x=[0,0,w,w], y=[0,h,h,0], fill="toself", 
                         name='Choose H1 & H1')) #name=P(Choose H1 & H1)/P(H1) = 1-β
fig.add_trace(go.Scatter(x=[0,0,h0_w*w, h0_w*w], y=[0,h,h,0], fill="toself", 
                         name='Choose H1 & H0')) #name=', P(Choose H1 & H0)/P(H0) = α'
fig.add_trace(go.Scatter(x=[0,0,h0_w*w, h0_w*w], y=[0,alpha*h,alpha*h,0], fill="toself", 
                         name='Choose H0 & H0')) #, P(Choose H0 & H0)/P(H0) = 1-α
fig.add_trace(go.Scatter(x=[h0_w*w,h0_w*w,w,w], y=[0,beta*h,beta*h,0], fill="toself", 
                         name='Choose H0 & H1')) # , P(Choose H0 & H1)/P(H1) = β
fig.add_annotation(x=h0_w/2*w, y=1.1*h,
            text="H0",
            showarrow=False)
fig.add_annotation(x=w/2 + h0_w/2*w, y=1.1*h,
            text="H1",
            showarrow=False)
fig.add_annotation(x=h0_w/2*w, y=(h+alpha*h)/2,
            text="Choose H1 & H0",
            showarrow=False)
fig.add_annotation(x=h0_w/2*w, y=beta*h/2,
            text="Choose H0 & H0",
            showarrow=False)
fig.add_annotation(x=(h0_w*w + w)/2, y=(h+alpha*h)/2,
            text="Choose H1 & H1",
            showarrow=False)
fig.add_annotation(x=(h0_w*w + w)/2, y=beta*h/2,
            text="Choose H0 & H1",
            showarrow=False)
fig.update_layout(
    height=450, width=800
)
fig.show()

По графику видно, что вероятность нулевой гипотезы если было принято решение ее оставить, определяется соотношением

$$
P(H_0 | \mbox{Choose }H_0) = \frac{P(\mbox{Choose }H_0 \cap H_0)}{P(\mbox{Choose }H_0 \cap H_0) + P(\mbox{Choose }H_0 \cap H_1)} = \frac{\mbox{Green}}{\mbox{Green} + \mbox{Purple}} .
$$

Вероятность графически вычисляется как отношение площадей соответствующих прямоугольников. Их высоты определяются значениями $\alpha$ и $\beta$. Но площади также зависит от ширины - т.е. от соотношения между вероятностями $P(H_0)$ и $P(H_1)$. Ошибка базового процента состоит в том, что решение принимается только по уровню $\alpha$ без учета соотношения $P(H_0)$ и $P(H_1)$.

Для характеристики метода можно оценить вероятность корректного выбора гипотезы. Ее можно выразить через вероятности ошибок $\alpha$ и $\beta$

\begin{align}
P(\mbox{Correct Guess}) & = P(\mbox{Choose }H_0 | H_0) P(H_0) + P(\mbox{Choose }H_1 | H_1)P(H_1) 
\\
    & = (1 - \alpha) P(H_0) + (1 - \beta) P(H_1) .
\end{align}

Видно, что доля правильно выбранных гипотез помимо $\alpha$ и $\beta$ зависит также от соотношения между $P(H_0)$ и $P(H_1)$. Если $P(H_1) \gg P(H_0)$, т.е. команда предлагает хорошие гипотезы, то $P(\mbox{Correct Guess}) \approx 1 - \beta$, если же гипотезы в-основном плохие $P(H_0) \gg P(H_1)$, то $P(\mbox{Correct Guess}) \approx 1 - \alpha$.

Хотя в реальном тесте неизвестно, верна гипотеза или нет, соотношение выше можно продемонстрировать на примере. Пусть есть 2 группы. В контрольной среднее значение $\mu_A = 0.1$ фиксировано. В экспериментальной среднее случайно выбирается из двух вариантов: либо такое же как в контрольной $\mu_B = \mu_A$, либо с улучшением на 5%  $\mu_B = 1.05 \mu_A$. Первый вариант соответствует реализации нулевой гипотезы $H_0$ о равенстве групп, второй - реализации гипотезы $H_1$ об их различии. Нужно оценить, с какой вероятностью в А/Б-тесте будет выбрана корректная гипотеза. 

Речь идет не о выборе "лучшей группы" - в этом примере всегда можно выбирать группу B - а о корректном выборе между вариантами $\mu_A = \mu_B$ и $\mu_A \ne \mu_B$.

На графике построена доля правильно выбранных гипотез при изменении соотношения между вероятностями $P(\mu_B = \mu_A)$ и $P(\mu_B = 1.05\mu_A)$, т.е. между $P(H_0)$ и $P(H_1)$. Можно ожидать, что при фиксированных $\alpha$ и $\beta$ доля будет меняться как $(1 - \alpha) P(H_0) + (1 - \beta) P(H_1)$.

При $\alpha = 0.05$, $\beta = 0.2$, $\Delta = 0.05$, $\mu_A = 0.1$ размер выборки в каждой группе $N \approx 57000$.

In [ ]:
mu0 = 0.1
mu1 = mu0 * 1.05
delta = mu1 - mu0

alpha = 0.05
beta = 0.2

ph0_prob = np.arange(0.01, 1.01, 0.1)
ph1_prob = 1 - ph0_prob

Nexp = 10000
corrguess_pval = []
corrguess_pval_theory = []
beta_pval = []
beta_pval_theory = []
alpha_pval = []
alpha_pval_theory = []
for ph0, ph1 in zip(ph0_prob, ph1_prob):    
    mua = np.full(shape=Nexp, fill_value=mu0)
    mub = np.random.choice(a=[mu0, mu1], p=[ph0, ph1], size=Nexp)
    N = int((np.sqrt(2) * (stats.norm.ppf(1 - alpha/2) - stats.norm.ppf(beta)))**2 * mu0*(1-mu0) / delta**2)
    sa = stats.binom.rvs(n=N, p=mua, size=Nexp)
    sb = stats.binom.rvs(n=N, p=mub, size=Nexp)
    pval = approx_ttest_conv_pval(sa, N, sb, N)
    h0 = (mub == mu0)
    h1 = (mub != mu0)
    reject_h0 = (pval < alpha/2) | (pval > 1 - alpha/2)
    keep_h0 = ~reject_h0
    corrguess_pval_theory.append((1-alpha)*ph0 + (1-beta)*ph1)
    corrguess_pval.append(sum(((keep_h0 & h0) | (reject_h0 & h1)))/Nexp)
    beta_pval.append(sum((keep_h0 & h1))/sum(h1))
    beta_pval_theory.append(beta)
    alpha_pval.append(sum((reject_h0 & h0))/sum(h0))
    alpha_pval_theory.append(alpha)
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=ph0_prob, y=corrguess_pval, name='Correct guesses'))
fig.add_trace(go.Scatter(x=ph0_prob, y=corrguess_pval_theory, line_dash='dash',
                         name='Correct guesses, theory'))
fig.add_trace(go.Scatter(x=ph0_prob, y=beta_pval, name='β'))
fig.add_trace(go.Scatter(x=ph0_prob, y=beta_pval_theory, line_dash='dash',
                         name='β, theory'))
fig.add_trace(go.Scatter(x=ph0_prob, y=alpha_pval, name='α'))
fig.add_trace(go.Scatter(x=ph0_prob, y=alpha_pval_theory, line_dash='dash',
                         name='α, theory'))
fig.update_layout(
    title='Correctly Chosen Hypotheses and Errors Rates',
    yaxis_title='Rate',
    xaxis_title='P(H0)',
    yaxis_range=[0, 1],
    width=800, height=450
)
fig.show()

Доля правильно выбранных гипотез и уровни ошибок $\alpha$ и $\beta$ такие, как и ожидалось. Видно, что доля зависит от соотношения $P(H_0):P(H_1)$. 

Можно сравнить количество правильно выбранных гипотез при выборе на основе $p$-значения и вероятности $P(H_0 | data)$. Т.к. гипотез всего две, то можно смотреть отношение $P(H_0 | data):P(H_1 | data)$. Если оно больше $1$, то выбирать $H_0$, если меньше - $H_1$. На основе соотношения Байеса

$$
\frac{P(H_0 | data)}{P(H_1 | data)} = \frac{P(data | H_0) P(H_0)}{P(data | H_1) P(H_1)}.
$$

В данном примере $P(data|H_1)$ можно вычислять как $P(data | \mu_B = 1.05 \mu_A)$. На практике разница между группами неизвестна, поэтому корректнее было бы использовать $P(data|\mu_A \ne \mu_B)$.

In [ ]:
def ph0_to_ph1_conv(sa, na, sb, nb, ph0, ph1, delta):
    pa = sa / na
    pb = sb / nb
    stderr_a = np.sqrt(pa * (1 - pa) / na)
    stderr_b = np.sqrt(pb * (1 - pb) / nb)
    diff = pb - pa
    diff_stderr = np.sqrt(stderr_a**2 + stderr_b**2)
    return stats.norm.pdf(diff, 0, diff_stderr) / stats.norm.pdf(diff, delta, diff_stderr) * ph0 / ph1

mu0 = 0.1
mu1 = mu0 * 1.05
delta = mu1 - mu0

alpha = 0.05
beta = 0.2

ph0_prob = []
cg_prob = []
cg_prob_factdelta = []
cg_pval = []
cg_pval_theory = []
for ph0 in np.arange(0.01, 1.01, 0.1):
    ph1 = 1-ph0
    ph0_prob.append(ph0)    
    Nexp = 10000
    mua = np.array([mu0] * Nexp)
    mub = np.random.choice(a=[mu0, mu1], p=[ph0, ph1], size=Nexp)
    N = int((np.sqrt(2) * (stats.norm.ppf(1 - alpha/2) - stats.norm.ppf(beta)))**2 * mu0*(1-mu0) / delta**2)
    sa = stats.binom.rvs(n=N, p=mua, size=Nexp)
    sb = stats.binom.rvs(n=N, p=mub, size=Nexp)
    pval = approx_ttest_conv_pval(sa, N, sb, N)
    p_h0_to_h1_data = ph0_to_ph1_conv(sa=sa, na=N, sb=sb, nb=N, ph0=ph0, ph1=ph1, delta=delta)
    h0_data = p_h0_to_h1_data > 1
    h1_data = ~h0_data
    h0 = (mub == mu0)
    h1 = (mub != mu0)
    reject_h0 = (pval < alpha/2) | (pval > 1 - alpha/2)
    keep_h0 = ~reject_h0
    cg_pval_theory.append((1-alpha)*ph0 + (1-beta)*ph1)
    cg_pval.append(sum(((keep_h0 & h0) | (reject_h0 & h1)))/Nexp)
    cg_prob.append(sum(((h0_data & h0) | (h1_data & h1)))/Nexp)

    
fig = go.Figure()
fig.add_trace(go.Scatter(x=ph0_prob, y=cg_prob, name='P(H0)'))
fig.add_trace(go.Scatter(x=ph0_prob, y=cg_pval, name='p-val'))
fig.add_trace(go.Scatter(x=ph0_prob, y=cg_pval_theory, line_dash='dash',
                         name='p-val theory'))
fig.update_layout(
    title='Correct Guesses by p-val and P(H0)',
    yaxis_title='Prob',
    xaxis_title='P(H0)',
    yaxis_range=[0, 1],
    width=800, height=450
)
fig.show()

В данном случае выбор группы по вероятности работает лучше. Это отчасти обусловлено тем, что модель использует дополнительную информацию о точном значении $\Delta$, неиспользуемую в проверке гипотез.

*В примере выше было известно отношение $P(H_0):P(\neg H_0)$ и для $P(data|H_1)$ можно было ограничиться вероятностью $P(data | \mu_B = 1.05 \mu_A)$. На практике соотношение $P(H_0):P(\neg H_0)$ и размер эффекта неизвестны. Для $P(H_0):P(\neg H_0)$ можно предположить значение, например 3:7. В ситуации с заранее неизвестным значением эффекта $\Delta$ попытки посчитать $P(data| \neg H_0)$ будут сдвигать подход в сторону байесовского моделирования. Проще делать все в рамках этого моделирования.*

Метод проверки гипотез формально не дает ответ на вопрос о "величине эффекта" и оценках "целевой метрики в каждом варианте". Для оценки средних вместо распределений $P(\mu_A - \mu_B | data)$, $P(\mu_A | data)$, $P(\mu_B | data)$ обычно используют доверительные интервалы [[ConfInt](https://en.wikipedia.org/wiki/Confidence_interval)]. Например, для $\mu_A$ строятся величины $l_n$, $u_n$, такие что

$$
l_n, u_n: P(l_n < \mu_A < u_n | data) = 1 - \alpha .
$$

Для средних границы интервала $l_n$, $u_n$ строят на основе центральной предельной теоремы   
*поправить*

$$
S_n = \xi_1 + ... + \xi_n
\\
a = E[\xi],
\qquad
\sigma^2 = D[\xi]
\\
P\left(\alpha < \frac{S_n - na}{\sqrt{n}\sigma} < \beta \right) \to \frac{1}{2 \pi} \int_{\alpha}^{\beta} e^{-t^2/2} dt 
\\
P \left( 
    x_n - \frac{c \sigma}{\sqrt{n}} < \mu < x_n + \frac{c \sigma}{\sqrt{n}}
\right) = \alpha
$$

В доверительных интервалах случайные величины - границы интервала, а не сам оцениваемый параметр.

В байесовском моделировании конверсий с биномиальным распределением как функцией правдоподобия и равномерным априорным распределением апостериорное распределение будет бета-распределением $P(x | data) = Beta(x; \alpha, \beta)$, $\alpha = s + 1$, $\beta = N - s + 1$. При большом количестве точек оно приближенно совпадает с нормальным

*проверить*

$$
P(x | data) \sim Norm\left( x; p, \frac{p(1-p)}{N} \right) .
$$

Поэтому интервалы наибольшей плотности вероятности, получаемые в рамках байесовского моделирования, численно будут близки к доверительным интервалам.

Доверительные интервалы не гарантируют, что "с 95% вероятностью значение $\mu_A$ будет внутри построенного интервала." Частотная интерпретация доверительных итервалов - если повторить эксперимент и процедуру построения 95-процентного доверительного интервала 100 раз, примерно в 95 случаях из 100 реальное среднее будет находится внутри построенного интервала (см. график ниже).  

In [ ]:
p = 0.6

n_exp = 100
N = 1000
s = stats.binom.rvs(n=N, p=p, size=n_exp)
p_mean = s / N
p_stderr = np.sqrt(p_mean * (1 - p_mean) / N)
alpha = 0.05
x_a_l = stats.norm.ppf(alpha/2, loc=p_mean, scale=p_stderr)
x_a_u = stats.norm.ppf(1-alpha/2, loc=p_mean, scale=p_stderr)

fig = go.Figure()
fig.add_hline(p)
missed = 0
for x, l, u in zip(range(1, N+1), x_a_l, x_a_u):
    col = 'black'
    if u < p or l > p:
        col = 'blue'
        missed += 1
    fig.add_trace(go.Scatter(x=[x,x], y=[l, u], mode='lines+markers', 
                             line_color=col))
fig.update_layout(
    title='Confidence Intervals',
    xaxis_title='Experiment #',
    yaxis_title='p',
    showlegend=False,
    height=450, width=800
)
fig.show()

print(f"In {n_exp} experiments, {missed} {1-alpha}-intervals failed to cover exact mean value.")

На графике выше видно, что по отдельно взятому доверительному интервалу нельзя сказать, где именно в нем находится среднее и попадает ли оно в этот интервал. Байесовский интервал наибольше плотности вероятности сталкивается с той же проблемой.

Хотя для средних доверительные интервалы совпадают с байесовскими интервалами наибольшей плотности вероятности, в других случаях они могут отличаться [[ConfIntVsBsInt](https://bayes.wustl.edu/etj/articles/confidence.pdf)]. *Еще обсуждение [[CIFal](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4742505/pdf/13423_2015_Article_947.pdf)].* 

Таким образом, основные сложности с доверительными интервалами для средних технические - используется две случайных величины вместо одной и для них нет распределений в явном виде.

Для оценки "длительности эксперимента" размер выборки рассчитывают до эксперимента так, чтобы обеспечить уровни статистической значимости $\alpha$ и мощности $\beta$. Задают статистическую значимость, мощность, базовую величину и изменение эффекта, по этим значениям рассчитывают размер выборки $N$ в каждой группе.  

Для $t$-тестов при оценке размера выборки находят точку $x_{\alpha \beta}$ такую, что $P(x \ge x_{\alpha \beta} | H_0) = \alpha$ и $P(x \le x_{\alpha \beta} | H_{\Delta}) = \beta$.

In [ ]:
mu = 0
scale = 0.2
delta = 0.5

x = np.arange(-3, 3, 0.01)
y0 = stats.norm.pdf(x, loc=mu, scale=scale)
y1 = stats.norm.pdf(x, loc=mu+delta, scale=scale)

a = 0.05
x_ab = stats.norm.ppf(1-alpha, loc=mu, scale=scale)

col0 = 'rgba(250, 0, 0, 0.7)'
col1 = 'rgba(0, 0, 250, 0.7)'

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y1, line_color=col1, name='HΔ'))
fig.add_trace(go.Scatter(x=x[x<=x_ab], y=y1[x<=x_ab], 
                         fill='tozeroy', 
                         line_color=col1, fillcolor=col1,
                         name='β'))
fig.add_vline(x_ab)
fig.add_trace(go.Scatter(x=x, y=y0, line_color=col0, name='H0'))
fig.add_trace(go.Scatter(x=x[x>=x_ab], y=y0[x>=x_ab], 
                         fill='tozeroy', 
                         line_color=col0, fillcolor=col0,
                         name='α'))
fig.update_layout(
    xaxis_range=(-2, 2),
    height=450, width=800
)

fig.show()

Выражение для размера сэмпла $N$

$$
\begin{cases}
\Phi \left( \frac{x_{\alpha \beta} - \mu_{H_0}}{\sigma_{H_0}} \right) = 1 - \alpha
\\
\Phi \left( \frac{x_{\alpha \beta} - \mu_{H_\Delta}}{\sigma_{H_{\Delta}}} \right) = \beta 
\end{cases}
\Rightarrow
\mu_{H_0} + \sigma_{H_0} \Phi^{-1}(1 - \alpha) = \mu_{H_\Delta} + \sigma_{H_{\Delta}}\Phi^{-1}(\beta),
\\
\mu_{H_0} = 0, 
\quad 
\mu_{H_\Delta} = \Delta,
\quad
\sigma_{H_0} = \sigma_{H_{\Delta}} \approx \sqrt{2}\frac{s}{\sqrt{N}},
\\
N = \left[ \frac{s}{\Delta} \sqrt{2} \left( \Phi^{-1}(1 - \alpha) - \Phi^{-1}(\beta) \right) \right]^2 .
$$

In [ ]:
def approx_t_sample_size(stderr, delta, alpha, beta):
    c = np.sqrt(2) * (stats.norm.ppf(1 - alpha) - stats.norm.ppf(beta))
    return np.ceil((c * stderr / delta)**2).astype(int)

p = 0.1
p_stderr = np.sqrt(p * (1-p))
approx_t_sample_size(stderr=p_stderr, delta=0.005, alpha=0.025, beta=0.2)

При $(1 - \alpha) = 0.975$, $\beta = 0.2$ приближенно [[StPower](https://en.wikipedia.org/wiki/Power_of_a_test#Rule_of_thumb)]

$$
N = 16 \frac{s^2}{\Delta^2} . 
$$

Проблема такого подхода в том, что размер эффекта $\Delta$ неизвестен. *Если $\mu_B \gt \mu_A + \Delta$, то $\beta$ будет меньше заданного (на графике выше средние будут правее максимума правого пика $H_1$). При этом будет переоценивается размер сэмпла. Если $\mu_B \le \mu_A + \Delta$, то $\beta$ будет выше заданного.*

*Можно оценить размер эффекта по части данных, после чего задать величину эффекта. Например, собрать по 100 точек, после чего выбрать размер эффекта на нижней границе доверительного интервала разности.*

*Возможным вариантом было бы формулировать гипотезы в виде $H_0: |\mu_A - \mu_B| \le \Delta$. Но это приближает подход к байесовскому моделированию.*

Для выполнения гарантий по $\alpha$ и $\beta$ требуется принимать решение об эксперименте по выборке определенного размера. При принятии решений по выборке меньшего размера гарантии на $\alpha$ и $\beta$ будут отозваны - это называют "проблемой подглядывания" [[MillerHowNotTo](https://www.evanmiller.org/how-not-to-run-an-ab-test.html)].

Выше при моделировании числа корректно угаданных вариантов было показано, что описанный способ оценки размера выборки действительно дает заданные значения $\alpha$ и $\beta$ когда есть только 2 варианта. Оценка специфична для $t$-теста. Для других тестов оценки должны делаться из других соображений.  

# Заключение

Приведен обзор метода проверки статистических гипотез. Рассмотрено использование $t$-тестов для сравнения средних в группах.

Принятие решения о выборе варианта на основе $p$-значения не дает ответа на вопрос какая группа лучше и какая в этом уверенность. Кроме того, принятие решения без учета альтернативных гипотез и их вероятностей является вариантом ошибки базового процента.

При сравнении средних для оценки метрик в группах и величины эффекта можно использовать доверительные интервалы на основе центральной предельной теоремы. Для конверсий при больших размерах выборки доверительные интервалы для средних численно будут близки к байесовским интервалам наибольшей плотности вероятности.  

Для оценки длительности эксперимента размер выборки оценивается так, чтобы "ошибки первого и второго рода" не превышали заданных уровней. Метод расчета свой для каждого теста. На практике размер эффекта заранее неизвестен.

Последовательное решение этих вопросов будет сдвигать подход в сторону байесовского моделирования. 

# Благодарности

# Ссылки

[[StTest](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing)] - Statistical hypothesis testing, *Wikipedia.*  
[[TestStat](https://en.wikipedia.org/wiki/Test_statistic)] - Test statistic, *Wikipedia.*  
[[TailedTests](https://en.wikipedia.org/wiki/One-_and_two-tailed_tests)] - One- and two-tailed tests, *Wikipedia.*   
[[PVal](https://en.wikipedia.org/wiki/P-value)] - P-value, *Wikipedia.*  
[[FairCoin](https://en.wikipedia.org/wiki/Checking_whether_a_coin_is_fair)] - Checking whether a coin is fair, *Wikipedia.*   
[[StErrors](https://en.wikipedia.org/wiki/Type_I_and_type_II_errors)] - Type I and type II errors, *Wikipedia.*   
[[StSign](https://en.wikipedia.org/wiki/Statistical_significance)] - Statistical significance, *Wikipedia.*  
[[StPower](https://en.wikipedia.org/wiki/Power_of_a_test)] - Power of a test, *Wikipedia.*  
[[HNull](https://en.wikipedia.org/wiki/Null_hypothesis)] - Null hypothesis, *Wikipedia.*  
[[StdErr](https://en.wikipedia.org/wiki/Standard_error)] - Standard error, *Wikipedia.*   
[[CLT](https://en.wikipedia.org/wiki/Central_limit_theorem)] - Central limit theorem, *Wikipedia.*   
[[TDist](https://en.wikipedia.org/wiki/Student%27s_t-distribution)] - Student's t-distribution, *Wikipedia.*     
[[TTest](https://en.wikipedia.org/wiki/Student's_t-test)] - Student's t-test, *Wikipedia.*      
[[WelchTTest](https://en.wikipedia.org/wiki/Welch%27s_t-test)] - Welch's t-test, *Wikipedia.*  
[[MillerABSize](https://www.evanmiller.org/ab-testing/sample-size.html)] - E. Miller, Sample Size Calculator, *Evan's Awesome A/B Tools.*     
[[SciPyT](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)] - scipy.stats.ttest_ind, *SciPy Reference.*    
[[BerEsTheor](https://en.wikipedia.org/wiki/Berry%E2%80%93Esseen_theorem)] - Berry-Esseen theorem, *Wikipedia.*   
[[KSTest](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test)] - Kolmogorov-Smirnov test, *Wikipedia.*  
[[UTest](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test)] - Mann-Whitney U-test, *Wikipedia.*   
[[MedianTest](https://en.wikipedia.org/wiki/Median_test)] - Median test, *Wikipedia.*    
[[StatTestMisinterpret](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4877414/)] S. Greenland et al., Statistical tests, P values, confidence intervals, and power: a guide to misinterpretations, *Eur J Epidemiol.*, 2016.  
[[UU](https://onlinelibrary.wiley.com/doi/book/10.1002/9781118650158)] - D.V. Lindley, *Understanding Uncertainty,* 2013.    
[[BaseRateFal](https://en.wikipedia.org/wiki/Base_rate_fallacy)] - Base rate fallacy, *Wikipedia.*  
[[ConfInt](https://en.wikipedia.org/wiki/Confidence_interval)] - Confidence interval, *Wikipedia.*   
[[ConfIntVsBsInt](https://bayes.wustl.edu/etj/articles/confidence.pdf)] - E.T. Jaynes, Confidence Intervals vs Bayesian Intervals, in *Foundations of Probability Theory, Statistical Inference, and Statistical Theories of Science*, 1976.    
[[CIFal](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4742505/pdf/13423_2015_Article_947.pdf)] -   
[[MillerHowNotTo](https://www.evanmiller.org/how-not-to-run-an-ab-test.html)] - E. Miller, How not to run an A/B test, *www.evanmiller.org*, 2010.    